In [17]:
import warnings
warnings.filterwarnings("ignore")
import dbutils
import time
import pandas as pd
from IPython.display import display
last_check = 0

In [8]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT DISTINCT(type) FROM artifact_quality WHERE type='20190708-0919_2379-595height';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result)

[('20190708-0919_2379-595height',)]


In [52]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE type='20190708-0919_2379-595height';"
result = main_connector.execute(sql_statement, fetch_one=True)[0]
print(last_check, result, "PCDs in artifact_quality")

if last_check != result:
    print("Change")
else:
    print("No change")
last_check = result

132565 132565 PCDs in artifact_quality
No change


In [7]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result, "PCDs in artifact")

[(223249,)] entries in artifact_quality


# Scan level MAEs.

In [5]:
main_connector = dbutils.connect_to_main_database()

sql_statement = ""
sql_statement += "SELECT COUNT(DISTINCT p.qr_code) FROM person p;"

result = main_connector.execute(sql_statement, fetch_one=True)[0]
print("Found {} qr-codes.".format(result))

Found 6859 qr-codes.


In [9]:
# Select all QR-code-create-date pairs. Those are scans.
sql_statement = ""
sql_statement += "SELECT DISTINCT p.qr_code, m.timestamp, m.id as measure_id"
sql_statement += " FROM measure m"
sql_statement += " INNER JOIN person p on m.person_id=p.id"
sql_statement += " WHERE m.type='manual'"
sql_statement += ";"

start = time.time()
scans = main_connector.execute(sql_statement, fetch_all=True)
end = time.time()
print(end - start)

print("Found {} manual scans".format(len(scans)))
#print(scans[200:300])

0.22005844116210938
Found 7028 scans


In [50]:
model = "20190708-0919_2379-595height"

for scan in scans[200:201]:
    qr_code = pair[0]
    create_date = pair[1]
    print(qr_code, create_date)
    
    sql_statement = ""
    sql_statement += "SELECT SUM(aq.value) "
    sql_statement += " FROM artifact_quality aq"
    sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
    sql_statement += " WHERE aq.key='mae'"
    sql_statement += " AND aq.type='{}'".format(model)
    sql_statement += " AND a.qr_code='{}'".format(qr_code)
    sql_statement += " AND a.create_date='{}'".format(create_date)
    sql_statement += ";"

    start = time.time()
    result = main_connector.execute(sql_statement, fetch_all=True)
    end = time.time()
    print(end - start)

MH_WHH_0077 1538717985089
9.917351961135864
9.407123565673828


In [51]:
6697 * 10 / (60 * 60)

18.602777777777778

In [25]:
print(qr_codes[0])
for qr_code in qr_codes[0:10]:
    qr_code = qr_code[0]
    
    # Get all create dates for that qr-code.
    sql_statement = ""
    sql_statement += "SELECT DISTINCT(a.create_date) FROM artifact a"
    sql_statement += " WHERE a.qr_code='{}'".format(qr_code)
    sql_statement += ";"
    
    start = time.time()
    create_dates = main_connector.execute(sql_statement, fetch_all=True)
    for create_date in create_dates:
        create_date = create_dates[0]
        print(r)
    end = time.time()
    print(end - start)
    break
    

('MH_WHH_0187',)
(1537851618537,)
11.59768009185791


In [22]:
#SELECT a.qr_code, AVG(aq.value) 
#FROM artifact_quality as aq
#INNER JOIN artifact as a
#  ON a.artifact_id = aq.artifact_id
#WHERE aq.key='mae'
#AND aq.type='20190708-0919_2379-595height'
#GROUP BY a.qr_code, a.create_date;

#main_connector = dbutils.connect_to_main_database()

model_name = "20190708-0919_2379-595height"

sql_statement = ""
sql_statement += "SELECT a.qr_code, a.create_date, AVG(aq.value) FROM artifact_quality aq"
sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
sql_statement += " WHERE aq.key='mae'"
sql_statement += " AND aq.type='{}'".format(model_name)
sql_statement += " GROUP BY a.qr_code, a.create_date"
sql_statement += ";"

print(sql_statement)

start = time.time()
result = main_connector.execute(sql_statement, fetch_all=True)
end = time.time()
print(end - start)


print(len(result), "results")

#last_check = len(result)
rows = []
for r in result[0:10]:
    qr_code = r[0]
    create_date = r[1]
    mae = r[2]
    row = (qr_code, create_date, mae)
    print(row)
    rows.append(row)
    
columns = ["QR-Code", "Timestamp", "MAE"]

df = pd.DataFrame(rows, columns=columns)
display(df)

SELECT a.qr_code, a.create_date, AVG(aq.value) FROM artifact_quality aq INNER JOIN artifact a ON a.id = aq.artifact_id WHERE aq.key='mae' AND aq.type='20190708-0919_2379-595height' GROUP BY a.qr_code, a.create_date;
5.318471670150757
4063 results
('MH_WHH_0001', 1536913319075, 8.64329643249512)
('MH_WHH_0002', 1536910804577, 0.409949730182516)
('MH_WHH_0003', 1536913710517, 15.8390997524919)
('MH_WHH_0007', 1537269381827, 9.00559854507446)
('MH_WHH_0007', 1537269534213, 9.87686772500315)
('MH_WHH_0010', 1537166990387, 10.1984226829127)
('MH_WHH_0010', 1537167038624, 9.68606387867647)
('MH_WHH_0011', 1537174095389, 1.1296624077691)
('MH_WHH_0012', 1536921402623, 8.35233191287879)
('MH_WHH_0013', 1537346511739, 5.59329244252798)


,QR-Code,Timestamp,MAE
0,MH_WHH_0001,1536913319075,8.643296
1,MH_WHH_0002,1536910804577,0.409950
2,MH_WHH_0003,1536913710517,15.839100
3,MH_WHH_0007,1537269381827,9.005599
4,MH_WHH_0007,1537269534213,9.876868
5,MH_WHH_0010,1537166990387,10.198423
6,MH_WHH_0010,1537167038624,9.686064
7,MH_WHH_0011,1537174095389,1.129662
8,MH_WHH_0012,1536921402623,8.352332
9,MH_WHH_0013,1537346511739,5.593292


In [ ]:

sql_statement = "SELECT * FROM artifacts_with_targets WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='number_of_points';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result)

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact WHERE path LIKE '%scantimestamp%';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result)

sql_statement = "SELECT COUNT(*) FROM artifact WHERE path LIKE '% %';"
print(sql_statement)
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result)

sql_statement = "SELECT COUNT(*) FROM artifact WHERE path NOT LIKE '%scantimestamp%';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result)



In [ ]:
sql_statement = "SELECT id, tango_timestamp FROM artifact WHERE tango_timestamp = ROUND(tango_timestamp) AND type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

sql_statement = "SELECT id, tango_timestamp FROM artifact WHERE tango_timestamp != ROUND(tango_timestamp) AND type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

#for r in result[0:10]:
#    print(r[0], r[1])

In [ ]:
sql_statement = "SELECT id, tango_timestamp FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)
result = result[:100]
print(result[0:2])
for r in result:
    print(r[0], r[1])

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='confidence_avg';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifact quality KPIs")

sql_statement = "SELECT MIN(value) FROM artifact_quality WHERE key='number_of_points';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifact quality KPIs")

In [ ]:
main_connector = dbutils.connect_to_main_database()

sql_statement = "SELECT COUNT(*) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality")

sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE value=0.0;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality where value is 0.0")

sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='confidence_avg';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality with key confidence_avg")

sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='number_of_points';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality with key number_of_points")

sql_statement = "SELECT COUNT(DISTINCT(artifact_id)) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "distinct artifact_ids in artifact_quality")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "PCDs in artifacts")

sql_statement = "SELECT COUNT(*) FROM artifacts_with_targets WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "PCDs in artifacts_with_targets")

In [ ]:
#sql_statement = "DELETE FROM artifact_quality WHERE value=0.0;"
#result = main_connector.execute(sql_statement, fetch_all=True)[0][0]

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifacts_with_targets WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "PCDs in artifacts_with_targets")